In [ ]:
# CNN using keras / model ensemble

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis] # 2D -> 3D
x_test = x_test[..., tf.newaxis]
y_train = tf.keras.utils.to_categorical(y_train, 10) # one-hot encoding
y_test = tf.keras.utils.to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1), padding="same"),
    MaxPooling2D(),
    Dropout(0.3),
    Conv2D(64, 3, activation='relu', padding="same"),
    MaxPooling2D(),
    Dropout(0.3),
    Conv2D(128, 3, activation='relu', padding="same"),
    MaxPooling2D(),
    Dropout(0.3),
    Flatten(),
    Dense(625, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=15, batch_size=100, validation_split=0.2)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/15
480/480 [==============================] - 20s 41ms/step - loss: 0.3152 - accuracy: 0.8969 - val_loss: 0.0630 - val_accuracy: 0.9817
Epoch 2/15
480/480 [==============================] - 19s 40ms/step - loss: 0.0892 - accuracy: 0.9718 - val_loss: 0.0445 - val_accuracy: 0.9864
Epoch 3/15
480/480 [==============================] - 20s 41ms/step - loss: 0.0663 - accuracy: 0.9795 - val_loss: 0.0318 - val_accuracy: 0.9908
Epoch 4/15
480/480 [==============================] - 20s 42ms/step - loss: 0.0536 - accuracy: 0.9825 - val_loss: 0.0337 - val_accuracy: 0.9899
Epoch 5/15
480/480 [==============================] - 21s 44ms/step - loss: 0.0472 - accuracy: 0.9850 - val_loss: 0.0311 - val_accuracy: 0.9911
Epoch 6/15
480/480 [==============================] - 21s 44ms/step - loss: 0.0413 - accuracy: 0.9869 - val_loss: 0.0291 - val_accuracy: 0.9913
Epoch 7/15
480/480 [==============================] - 21s 43ms/step - loss: 0.0387 - accuracy: 0.9881 - val_loss: 0.0334 - val_accuracy:

In [2]:
import numpy as np

def create_and_train_model(x_train, y_train, x_val, y_val):
    model = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1), padding="same"),
        MaxPooling2D(),
        Dropout(0.3),
        Conv2D(64, 3, activation='relu', padding="same"),
        MaxPooling2D(),
        Dropout(0.3),
        Conv2D(128, 3, activation='relu', padding="same"),
        MaxPooling2D(),
        Dropout(0.3),
        Flatten(),
        Dense(625, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=15, batch_size=100, validation_data=(x_val, y_val))
    return model

num_models = 7
models = [create_and_train_model(x_train, y_train, x_test, y_test) for _ in range(num_models)]

predictions = [model.predict(x_test) for model in models]
predictions = np.mean(predictions, axis=0)

ensemble_accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
print("Ensemble accuracy:", ensemble_accuracy)


Epoch 1/15
600/600 [==============================] - 22s 37ms/step - loss: 0.2631 - accuracy: 0.9145 - val_loss: 0.0466 - val_accuracy: 0.9857
Epoch 2/15
600/600 [==============================] - 22s 37ms/step - loss: 0.0759 - accuracy: 0.9763 - val_loss: 0.0275 - val_accuracy: 0.9900
Epoch 3/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0573 - accuracy: 0.9821 - val_loss: 0.0274 - val_accuracy: 0.9917
Epoch 4/15
600/600 [==============================] - 24s 39ms/step - loss: 0.0482 - accuracy: 0.9847 - val_loss: 0.0224 - val_accuracy: 0.9932
Epoch 5/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0448 - accuracy: 0.9856 - val_loss: 0.0217 - val_accuracy: 0.9919
Epoch 6/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0374 - accuracy: 0.9874 - val_loss: 0.0242 - val_accuracy: 0.9929
Epoch 7/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0375 - accuracy: 0.9884 - val_loss: 0.0207 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 24s 40ms/step - loss: 0.2545 - accuracy: 0.9176 - val_loss: 0.0504 - val_accuracy: 0.9833
Epoch 2/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0771 - accuracy: 0.9763 - val_loss: 0.0307 - val_accuracy: 0.9894
Epoch 3/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0596 - accuracy: 0.9814 - val_loss: 0.0304 - val_accuracy: 0.9916
Epoch 4/15
600/600 [==============================] - 25s 41ms/step - loss: 0.0528 - accuracy: 0.9835 - val_loss: 0.0252 - val_accuracy: 0.9919
Epoch 5/15
600/600 [==============================] - 26s 43ms/step - loss: 0.0452 - accuracy: 0.9856 - val_loss: 0.0235 - val_accuracy: 0.9917
Epoch 6/15
600/600 [==============================] - 25s 41ms/step - loss: 0.0397 - accuracy: 0.9874 - val_loss: 0.0239 - val_accuracy: 0.9925
Epoch 7/15
600/600 [==============================] - 25s 41ms/step - loss: 0.0371 - accuracy: 0.9885 - val_loss: 0.0245 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 26s 42ms/step - loss: 0.2608 - accuracy: 0.9152 - val_loss: 0.0421 - val_accuracy: 0.9864
Epoch 2/15
600/600 [==============================] - 26s 43ms/step - loss: 0.0817 - accuracy: 0.9740 - val_loss: 0.0406 - val_accuracy: 0.9868
Epoch 3/15
600/600 [==============================] - 26s 43ms/step - loss: 0.0598 - accuracy: 0.9812 - val_loss: 0.0288 - val_accuracy: 0.9899
Epoch 4/15
600/600 [==============================] - 26s 43ms/step - loss: 0.0507 - accuracy: 0.9843 - val_loss: 0.0248 - val_accuracy: 0.9915
Epoch 5/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0434 - accuracy: 0.9863 - val_loss: 0.0203 - val_accuracy: 0.9926
Epoch 6/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0409 - accuracy: 0.9874 - val_loss: 0.0214 - val_accuracy: 0.9930
Epoch 7/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0355 - accuracy: 0.9887 - val_loss: 0.0213 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 26s 44ms/step - loss: 0.2649 - accuracy: 0.9136 - val_loss: 0.0448 - val_accuracy: 0.9844
Epoch 2/15
600/600 [==============================] - 26s 43ms/step - loss: 0.0774 - accuracy: 0.9756 - val_loss: 0.0417 - val_accuracy: 0.9862
Epoch 3/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0604 - accuracy: 0.9804 - val_loss: 0.0264 - val_accuracy: 0.9913
Epoch 4/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0505 - accuracy: 0.9842 - val_loss: 0.0254 - val_accuracy: 0.9911
Epoch 5/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0448 - accuracy: 0.9857 - val_loss: 0.0254 - val_accuracy: 0.9916
Epoch 6/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0397 - accuracy: 0.9879 - val_loss: 0.0228 - val_accuracy: 0.9936
Epoch 7/15
600/600 [==============================] - 26s 44ms/step - loss: 0.0359 - accuracy: 0.9886 - val_loss: 0.0198 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 23s 38ms/step - loss: 0.2676 - accuracy: 0.9128 - val_loss: 0.0459 - val_accuracy: 0.9850
Epoch 2/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0803 - accuracy: 0.9746 - val_loss: 0.0323 - val_accuracy: 0.9893
Epoch 3/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0581 - accuracy: 0.9815 - val_loss: 0.0297 - val_accuracy: 0.9901
Epoch 4/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0504 - accuracy: 0.9839 - val_loss: 0.0239 - val_accuracy: 0.9915
Epoch 5/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0438 - accuracy: 0.9859 - val_loss: 0.0272 - val_accuracy: 0.9911
Epoch 6/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0408 - accuracy: 0.9871 - val_loss: 0.0259 - val_accuracy: 0.9915
Epoch 7/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0359 - accuracy: 0.9888 - val_loss: 0.0226 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 24s 39ms/step - loss: 0.2687 - accuracy: 0.9126 - val_loss: 0.0490 - val_accuracy: 0.9842
Epoch 2/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0753 - accuracy: 0.9758 - val_loss: 0.0306 - val_accuracy: 0.9896
Epoch 3/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0569 - accuracy: 0.9822 - val_loss: 0.0259 - val_accuracy: 0.9920
Epoch 4/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0495 - accuracy: 0.9848 - val_loss: 0.0226 - val_accuracy: 0.9921
Epoch 5/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0423 - accuracy: 0.9869 - val_loss: 0.0217 - val_accuracy: 0.9927
Epoch 6/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0393 - accuracy: 0.9879 - val_loss: 0.0196 - val_accuracy: 0.9930
Epoch 7/15
600/600 [==============================] - 23s 39ms/step - loss: 0.0355 - accuracy: 0.9889 - val_loss: 0.0192 - val_accuracy:

Epoch 1/15
600/600 [==============================] - 24s 39ms/step - loss: 0.2602 - accuracy: 0.9147 - val_loss: 0.0493 - val_accuracy: 0.9844
Epoch 2/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0808 - accuracy: 0.9749 - val_loss: 0.0352 - val_accuracy: 0.9882
Epoch 3/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0623 - accuracy: 0.9812 - val_loss: 0.0275 - val_accuracy: 0.9894
Epoch 4/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0516 - accuracy: 0.9840 - val_loss: 0.0236 - val_accuracy: 0.9920
Epoch 5/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0457 - accuracy: 0.9855 - val_loss: 0.0232 - val_accuracy: 0.9914
Epoch 6/15
600/600 [==============================] - 23s 38ms/step - loss: 0.0390 - accuracy: 0.9877 - val_loss: 0.0214 - val_accuracy: 0.9931
Epoch 7/15
600/600 [==============================] - 24s 40ms/step - loss: 0.0361 - accuracy: 0.9890 - val_loss: 0.0210 - val_accuracy: